# Analisis de sentimientos

Creacion de un analizador de sentimientos a partir de texto, usando el conjunto de datos https://ai.stanford.edu/~amaas/data/sentiment/. Este conjunto consta de 2 carpetas, train y test, cada una de las cuales poseen a su vez dos carpetas, pos y neg, las que almacenan comentarios de peliculas, segun recuerdo, positivos y negativos respectivamente. La idea es entrenar un modelo que sea capaz de predecir si los comentarios son positivos o negativos. Teniendo en cuenta que las categorias de los comentarios son positivos o negativos, se hace uso de regresion logistica, la cual a su vez hace uso de la funcion sigmoide. La funcion sigmoide, segun recuerdo, tiene la forma f(x) = 1/ [1 + e^(-t) ], donde t en este caso sera el vector de palabras.


In [46]:
#Importamos las bibliotecas basicas
import os
import nltk
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [47]:
#Definimos la carpeta donde estan los conjuntos de entrenamiento y prueba respectivamente
train_folder, test_folder,encuesta_folder = '/home/mleon/Escritorio/PLN/aclImdb/train/', '/home/mleon/Escritorio/PLN/aclImdb/test/','/home/mleon/Escritorio/PLN/aclImdb/encuesta/'

In [48]:
"""
Creamos el conjunto de datos. Es decir, para el conjunto de entrenamiento, tomaremos todos los comentarios positivos y los
almacenaremos juntos y etiquetaremos como 1, por otra parte con los comentarios negativos igualmente los almacenaremos en
donde mismo pusimos los comentarios positivos del conjunto de entrenamiento, pero en este caso los etiquetaremos como 0.
"""
def creatingDataSet(folder, folder_type = ['pos/', 'neg/'], result_file_path = None):
    data = []
    
    for f_type in folder_type:
        temp_folder=folder + f_type
        print(temp_folder)
        folder_class = 1
        if f_type != 'pos/':
            folder_class = 0
        #Listando los archivos
        files = os.listdir(temp_folder)
        #Analizando los archivos
        for file in files:
            with open(temp_folder+file, "r", encoding="utf8") as f:
                value = f.read()
                data.append( [value, folder_class] )
    
    df = pd.DataFrame( data = data, columns = ['text', 'class'])
    if result_file_path:
        df.to_csv(result_file_path, index = False)
    return df


In [49]:
#Creando archivos CSV donde se unifican los datos de entrenamiento, asi como con los datos de prueba
creatingDataSet('/home/mleon/Escritorio/PLN/aclImdb/train/', result_file_path='/home/mleon/Escritorio/PLN/aclImdb/train.csv')
creatingDataSet('/home/mleon/Escritorio/PLN/aclImdb/test/', result_file_path='/home/mleon/Escritorio/PLN/aclImdb/test.csv')
creatingDataSet('/home/mleon/Escritorio/PLN/aclImdb/encuesta/', result_file_path='/home/mleon/Escritorio/PLN/aclImdb/encuesta.csv')

/home/mleon/Escritorio/PLN/aclImdb/train/pos/
/home/mleon/Escritorio/PLN/aclImdb/train/neg/
/home/mleon/Escritorio/PLN/aclImdb/test/pos/
/home/mleon/Escritorio/PLN/aclImdb/test/neg/
/home/mleon/Escritorio/PLN/aclImdb/encuesta/pos/
/home/mleon/Escritorio/PLN/aclImdb/encuesta/neg/


,text,class
0,"I liked the interview, I felt like in American...",1
1,It was difficult because I feel that I lack ti...,0


#  Vectorizando los conjuntos de entrenamiento y prueba



En esta fase convertiremos cada uno de los textos a vector, es decir a numeros, para su posterior analisis.


In [45]:


train_dataset = pd.read_csv('/home/mleon/Escritorio/PLN/aclImdb/train.csv')
test_dataset = pd.read_csv('/home/mleon/Escritorio/PLN/aclImdb/test.csv')




In [6]:


#Importamos la biblioteca que utilizaremos para la vectorizacion de las palabras
from sklearn.feature_extraction.text import TfidfVectorizer



In [7]:


"""
Descartamos las palabras cuya frecuencia es menor a .1 o superior a .5; es decir, descartamos las palabras con una baja
frecuencia, pues estas suelen ser errores ortograficos. Asimismo descartamos las palabras con una alta frecuencia, pues 
estas suelen ser preposiciones, articulos y conjunciones.
"""
tfidf = TfidfVectorizer(min_df = .1, max_df = .5, ngram_range=(1, 3), stop_words = 'english')



In [8]:


#Obteniendo los textos de entrenamiento y prueba a vectorizar
train_dataset_texts = train_dataset['text'].values
test_dataset_texts = test_dataset['text'].values



In [32]:


#Transformando los textos de entrenamiento
features = tfidf.fit_transform(train_dataset_texts)
#Ahora el text 
print(train_dataset_texts[1000])
#se traduce en
print(features.todense()[0])



This very strange movie is unlike anything made in the west at the time. With its tumultuous emotions and net of visions, dreams, and startling images, its effect is both beautiful and unsettling. The actors are choreographed more like dance than acting. It contains the only dream sequence I know of that actually resembles a real nightmare (sorry, Dali fans).
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.21936995 0.21527247
  0.         0.         0.         0.         0.         0.
  0.         0.22471577 0.         0.         0.         0.
  0.         0.         0.3244169  0.         0.         0.
  0.         0.         0.         0.         0.         0.21935281
  0.         0.49510277 0.         0.         0.19976217 0.
  0.         0.         0.         0.50645754 0.         0.
  0.         0.         0.         0.         0.24136148 0.18167004
  0.         0.         0.         0.         0.         0.
  0.         0

In [27]:
#Obteniendo las clases del conjunto de entrenamiento
train_dataset_class = train_dataset['class'].values
#Transformando las clases del conjunto de entrenamiento a vector columna
train_dataset_class = train_dataset_class.reshape( -1, 1)
print(train_dataset_class.shape)
#Transformando la matriz dispersa a matriz
train_mtx = features.todense()
print(train_mtx.shape)
train_mtx_array = np.array( train_mtx )
train_mtx_array = np.hstack( (train_mtx_array, train_dataset_class) )
print(train_mtx_array.shape)

#Exportando el dataframe
columns = tfidf.get_feature_names_out()
columns = columns.tolist()
columns.append('class')

df = pd.DataFrame( data = train_mtx_array, columns = columns)
df.to_csv('/home/mleon/Escritorio/PLN/aclImdb/train_vector.csv', index = False)

(25002, 1)
(25002, 77)
(25002, 78)


In [28]:


features = tfidf.transform(test_dataset_texts)



In [29]:
#Obteniendo las clases del conjunto de prueba
test_dataset_class = test_dataset['class'].values
#Transformando las clases del conjunto de prueba a vector columna
test_dataset_class = test_dataset_class.reshape( -1, 1)
print(test_dataset_class.shape)
#Transformando la matriz dispersa a matriz
test_mtx = features.todense()
print(test_mtx.shape)
test_mtx_array = np.array( test_mtx )
test_mtx_array = np.hstack( (test_mtx_array, test_dataset_class) )
print(test_mtx_array.shape)

#Exportando el dataframe
columns = tfidf.get_feature_names_out()
columns = columns.tolist()
columns.append('class')

df = pd.DataFrame( data = test_mtx_array, columns = columns)
df.to_csv('/home/mleon/Escritorio/PLN/aclImdb/test_vector.csv', index = False)


(25002, 1)
(25002, 77)
(25002, 78)


 # Logistic



Una vez los textos estan vectorizados paramos a la creacion del modelo


In [13]:


#Leemos los conjuntos de datos de entrenamiento y prueba
train_dataset = pd.read_csv('/home/mleon/Escritorio/PLN/aclImdb/train_vector.csv')
test_dataset = pd.read_csv('/home/mleon/Escritorio/PLN/aclImdb//test_vector.csv')



In [14]:
#Obtenemos los vectores de los comentarios, es decir, se ignora la ultima columna pues esta indica si el comentario fue
#positivo o negativo
X_train = train_dataset.iloc[:,:-1]
print(X_train.shape)
#Obtenemos la etiqueta de los comentarios
y_train = train_dataset['class'].values
print(y_train)

X_test = test_dataset.iloc[:,:-1]
print(X_test.shape)
y_test = test_dataset['class'].values
print(y_test)

(25002, 77)
[1. 1. 1. ... 0. 0. 0.]
(25002, 77)
[1. 1. 1. ... 0. 0. 0.]


In [15]:
#Creamos el modelo de regresion logistica
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
#Evaluamos la exactirud del modelo para etiquetar los comentarios presentes en el conjunto de entrenamiento
print('Train Score', clf.score(X_train, y_train))
#Evaluamos la exactirud del modelo para etiquetar los comentarios presentes en el conjunto de prueba
print('Test Score', clf.score(X_test, y_test))


Train Score 0.7173826093912486
Test Score 0.7156627469802416




Como vemos tenemos cerca de un 70 porciento de exactitud al etiquetar los comentarios.


# Hashing



Sin embargo, la anterior estrategia de vectorizacion no resulta del todo adecuada cuando trabajamos con conjuntos de datos demasiado extensos, en el orden de los terabytes, es decir, cuando el numero de caracteristicas es demasiado grande. En este momento una mejor estrategia es vectorizar las caracteristicas, de forma tal que para cada palabra cada caracter es multiplicado por un determinado numero primo elevado a la i-esima potencia, para i=0...no_caracteres - 1. Es decir, el hash de una cadena cualquiera es: hash(s) = s[0] + s[1] p^{1} + ... + s[n] p^{n-1}. Ahora vemos un ejemplo de lo anteriormente expuesto:


In [16]:


from sklearn.feature_extraction.text import HashingVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]
vectorizer = HashingVectorizer(n_features=2**4)
X = vectorizer.fit_transform(corpus)
print(X.shape)
X.todense()



(4, 16)


matrix([[-0.57735027,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        , -0.57735027,  0.        ,
          0.        ,  0.        ,  0.        ,  0.57735027,  0.        ,
          0.        ],
        [-0.81649658,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.40824829,  0.        ,  0.40824829,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        , -0.70710678,
          0.70710678,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [-0.57735027,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        , -0.57735027,  0.        ,
          0.        ,  0.        ,  0.        ,  0.57735027,  0.        ,
          0.        ]])



Creamos un vectorizador, el cual aplicara una funcion hash a cada palabra, ademas esta tabla hash constara de 1024 elementos, lo que si bien pudiese provocar que existan colisiones y disminuya la exactitud del modelo, en este caso no es asi. Aunque debemos recordar que en caso de que lo deseemos podemos aumentar el numero de caracteristicas hasta 2^20, lo que disminuye la posibilidad de que ocurran colisiones.


In [17]:


vectorizer = HashingVectorizer(n_features=2**10, stop_words = 'english')
features = vectorizer.fit_transform(train_dataset_texts)



In [18]:


#Obteniendo las clases del conjunto de entrenamiento
train_dataset_class = train_dataset['class'].values
#Transformando la matriz dispersa a matriz
train_mtx = features.todense()
print(train_mtx.shape)
train_mtx_array = np.array( train_mtx )

#Las 1024 caracteristicas que surgen son resultados de la aplicacion de una funcion hash, por lo que no se puede hacer el 
#proceso inverso, es decir, a partir de valor hash recuperar la caracteristica a la que esta asociada, pues en realidad 
#no es del todo asi. Motivo por el cual no se exporto los dataframe que resultaron de la vectorizacion.



(25002, 1024)


In [19]:


features = vectorizer.transform(test_dataset_texts)



In [20]:


#Obteniendo las clases del conjunto de prueba
test_dataset_class = test_dataset['class'].values
#Transformando las clases del conjunto de prueba a vector columna
test_dataset_class = test_dataset_class.reshape( -1, 1)
print(test_dataset_class.shape)
#Transformando la matriz dispersa a matriz
test_mtx = features.todense()
print(test_mtx.shape)
test_mtx_array = np.array( test_mtx )



(25002, 1)
(25002, 1024)


In [21]:
X_train = train_mtx_array
y_train = train_dataset_class

X_test = test_mtx_array
y_test = test_dataset_class


In [22]:
#Creamos el modelo de regresion logistica
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
#Evaluamos la exactirud del modelo para etiquetar los comentarios presentes en el conjunto de entrenamiento
print('Train Score', clf.score(X_train, y_train))
#Evaluamos la exactirud del modelo para etiquetar los comentarios presentes en el conjunto de prueba
print('Test Score', clf.score(X_test, y_test))

Train Score 0.8250939924806016
Test Score 0.7993360531157507


# Aplicandolo a las encuestas


In [40]:
Encuesta_dataset = pd.read_csv('/home/mleon/Escritorio/PLN/analisis/Respuestas/Guadalajara_Respuestas.csv')
from sklearn.feature_extraction.text import TfidfVectorizer
Encuesta_dataset_texts = Encuesta_dataset['text'].values



KeyError: 'text'

In [41]:
#Transformando los textos 
features = tfidf.fit_transform(Encuesta_dataset_texts)
#Ahora el text 
print(Encuesta_dataset_texts[1000])
#se traduce en
print(features.todense()[0])

NameError: name 'Encuesta_dataset_texts' is not defined